In [1]:
from langchain import hub
from langchain.schema import StrOutputParser, Document
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_ollama import OllamaLLM
from langchain.embeddings import HuggingFaceBgeEmbeddings
from concurrent.futures import ThreadPoolExecutor
from langchain.document_loaders.base import BaseLoader
from langchain.docstore.document import Document
from typing import List, Optional
from functools import lru_cache
from langchain.storage import InMemoryStore
import time
import re
import os
import pandas as pd

In [ ]:
query_1 = "What is the objective of the project with grant agreement 740934?"
query_2 = "What is the total cost of the project with the acronym HYPERGRYD?"
query_3 = "How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
query_4 = "Which organisation played the role of coordinator in the grant agreement 777998?"
query_5 = "What topic does the project with the acronym INTERRFACE belong to?"
query_6 = "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
query_7 = "What type of proposal was the grant agreement 814416?"
query_8 = "To which master call was the project with the acronym G9NIGHT submitted?"
query_9 = "To which sub call was the project titled Electron Nanocrystallography submitted?"
query_10 = "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
query_11 = "Provide the acronym of 1 project which objective is related to robotics."
query_12 = "Provide the title of 1 project which objective is related to geolocation."
query_13 = "Provide the objective of 1 project related to digital twin."
query_14 = "Provide the objective of 3 different projects related to corrosion."
query_15 = "Provide the title of 3 different projects which objective is related to offshore structures."
query_16 = "Provide the acronym of 3 different projects which objective is related to materials engineering."
query_17 = "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
query_18 = "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
query_19 = "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
query_20 = "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
query_21 = "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
query_22 = "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."
query_23 = "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."
query_24 = "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."
query_25 = "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

In [3]:
import pandas as pd
import re
import difflib

path_df = "exps 6-9/Experimento_7.xlsx"

df_principal = pd.read_excel(path_df)

archivo_excel = "Respuestas.xlsx"
excel = pd.ExcelFile(archivo_excel)

umbral_similitud = 0.8

resultados = []

for hoja in excel.sheet_names:
    numeros = re.findall(r'\d+', hoja)
    if not numeros:
        resultados.append({
            "Hoja": hoja,
            "Índice Hoja": "Sin número",
            "Fila en hoja": None,
            "Valor encontrado": None,
            "Similitud": None,
            "Coincidencia exacta": None,
            "Resultado": "Hoja sin número"
        })
        continue

    indice = int(numeros[0]) - 1

    if indice < 0 or indice >= len(df_principal):
        resultados.append({
            "Hoja": hoja,
            "Índice Hoja": indice + 1,
            "Fila en hoja": None,
            "Valor encontrado": None,
            "Similitud": None,
            "Coincidencia exacta": None,
            "Resultado": "Índice fuera de rango"
        })
        continue

    texto_respuesta = str(df_principal.loc[indice, "Answer"])

    try:
        df_hoja = excel.parse(hoja)
    except Exception as e:
        resultados.append({
            "Hoja": hoja,
            "Índice Hoja": indice + 1,
            "Fila en hoja": None,
            "Valor encontrado": None,
            "Similitud": None,
            "Coincidencia exacta": None,
            "Resultado": f"Error al leer hoja: {e}"
        })
        continue

    if df_hoja.empty:
        resultados.append({
            "Hoja": hoja,
            "Índice Hoja": indice + 1,
            "Fila en hoja": None,
            "Valor encontrado": None,
            "Similitud": None,
            "Coincidencia exacta": None,
            "Resultado": "Hoja vacía"
        })
        continue

    primera_columna = df_hoja.columns[0]

    encontrados = []

    for fila_idx, valor in df_hoja[primera_columna].dropna().astype(str).items():
        similitud = difflib.SequenceMatcher(None, valor.lower(), texto_respuesta.lower()).ratio()
        if valor in texto_respuesta or similitud >= umbral_similitud:
            encontrados.append({
                "Hoja": hoja,
                "Índice Hoja": indice + 1,
                "Fila en hoja": fila_idx,
                "Valor encontrado": valor,
                "Similitud": round(similitud, 3),
                "Coincidencia exacta": valor in texto_respuesta,
                "Resultado": "Coincidencia"
            })

    if encontrados:
        resultados.extend(encontrados)
    else:
        resultados.append({
            "Hoja": hoja,
            "Índice Hoja": indice + 1,
            "Fila en hoja": None,
            "Valor encontrado": None,
            "Similitud": None,
            "Coincidencia exacta": False,
            "Resultado": "Sin coincidencias"
        })

df_resultados = pd.DataFrame(resultados)

print(df_resultados)
# Guardar en Excel si quieres
# df_resultados.to_excel("coincidencias_completas.xlsx", index=False)


   Hoja  Índice Hoja  Fila en hoja    Valor encontrado  Similitud  \
0    10           10           NaN                None        NaN   
1    11           11           8.0               RODIN      0.101   
2    11           11          14.0                ODIN      0.082   
3    12           12           NaN                None        NaN   
4    15           15           NaN                None        NaN   
5    16           16           NaN                None        NaN   
6    17           17           NaN                None        NaN   
7    18           18           NaN                None        NaN   
8    19           19           NaN                None        NaN   
9    20           20           NaN                None        NaN   
10   21           21         900.0  AARHUS UNIVERSITET      0.188   
11   21           21         961.0  AARHUS UNIVERSITET      0.188   
12   21           21        1139.0  AARHUS UNIVERSITET      0.188   
13   22           22         124.0